## Build a Wflow model from scratch

This notebook demonstrates how to prepare **Wflow** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

Lets first check if the Wflow model is recognized by hydromt

In [1]:
# this should return "wflow, wflow_seciment"
!hydromt --models

hydroMT model plugins: wflow (v0.1.2.dev0), wflow_sediment (v0.1.2.dev0)


Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT REGION

  Build models from source data.

  Example usage:
  --------------

  To build a wflow model for a subbasin using and point coordinates snapped to cells with stream order >= 4
  hydromt build wflow /path/to/model_root "{'subbasin': [-7.24, 62.09], 'strord': 4}" -i /path/to/wflow_config.ini

  To build a wflow model based on basin ID
  hydromt build wflow /path/to/model_root "{'basin': 230001006}"

  To build a sfincs model based on a bbox (for Texel)
  hydromt build sfincs /path/to/model_root "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

Options:
  -r, --res FLOAT             Model resolution in model src.
  --build-base / --build-all  Deprecated!
  --opt TEXT                  Component specific keyword arguments, see the
                              setup_<component> method of the specific model
                              for more information about the arguments.

  -i, --config PATH           Path to hydroMT configuration fi

### setup wflow model base layers 

In [3]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_test_base" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -v

2021-06-11 15:19:08,711 - build - log - INFO - HydroMT version: v0.4.2.dev
2021-06-11 15:19:08,711 - build - main - INFO - Building instance of wflow model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_base.
2021-06-11 15:19:08,712 - build - main - INFO - User settings:


2021-06-11 15:19:08,756 - build - model_api - INFO - Initializing wflow model from hydromt_wflow (v0.1.2.dev0).
2021-06-11 15:19:08,757 - build - model_api - INFO - setup_basemaps.region: {'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7]}
2021-06-11 15:19:08,757 - build - model_api - INFO - setup_basemaps.res: 0.00833
2021-06-11 15:19:08,757 - build - model_api - INFO - setup_basemaps.hydrography_fn: merit_hydro
2021-06-11 15:19:08,757 - build - model_api - INFO - setup_basemaps.basin_index_fn: merit_hydro_index
2021-06-11 15:19:08,757 - build - model_api - INFO - setup_basemaps.upscale_method: ihu
2021-06-11 15:19:08,757 - build - wflow - INFO - Preparing base hydrography basemaps.
2021-06-11 15:19:08,758 - build - data_adapter - INFO - Downloading file to /home/runner/.hydromt_data/data/v0.0.4/data.tar.gz


2021-06-11 15:19:11,692 - build - data_adapter - INFO - Updating data sources from yml file /home/runner/.hydromt_data/data/v0.0.4/data_catalog.yml


2021-06-11 15:19:11,771 - build - data_adapter - INFO - DataCatalog: Getting merit_hydro RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.4/merit_hydro/{variable}.tif


2021-06-11 15:19:15,494 - build - basin_mask - INFO - basin bbox: [11.7750, 45.8042, 12.7450, 46.6900] / size: 664756


2021-06-11 15:19:15,770 - build - basemaps - INFO - Upscale flow direction data: 10x, ihu method.


2021-06-11 15:19:34,692 - build - wflow - INFO - Write model data to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_base
2021-06-11 15:19:34,695 - build - model_api - INFO - Writing model config to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_base/wflow_sbm.toml
2021-06-11 15:19:34,696 - build - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_base/staticmaps.nc


2021-06-11 15:19:34,823 - build - wflow - INFO - Writing model staticgeom to file.


The example above means the following: run **hydromt build** with:

* `wflow` : i.e. build a wflow model
* `./wflow_test_base` : output model folder
* `"{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-r 0.0083333` : build the model at a 30 arc sec (~1km) resolution
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (topography and hydrography) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html). 

We will load the default wflow build ini file for inspection:

In [4]:
fn_ini = 'wflow_build.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
data_libs       = []            # add optional paths to data yml files

[setup_config]                  # options parsed to wflow ini file <section>.<option>
starttime = 2010-01-01T00:00:00
endtime = 2010-03-31T00:00:00
timestepsecs = 86400
input.path_forcing = inmaps-era5-2010.nc

[setup_basemaps]
hydrography_fn     = merit_hydro   # source hydrography data {merit_hydro, merit_hydro_1k}
basin_index_fn  = merit_hydro_index # source of basin index corresponding to hydrography_fn
upscale_method  = ihu           # upscaling method for flow direction data, by default 'ihu'

[setup_rivers]
hydrography_fn  = merit_hydro   # source hydrography data, should match basemaps source
river_upa       = 30            # minimum upstream area threshold for the river map [km2]
slope_len       = 2000          # length over which tp calculate river slope [m]

[setup_reservoirs]
reservoirs_fn   = hydro_reservoirs  # source for reservoirs based on GRAND: {hydro_reservoirs}; None to skip
min_area   

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `--dd` flag. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete wflow model 

In [5]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_test_full" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -i wflow_build.ini -vv

2021-06-11 15:19:37,918 - build - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full/hydromt.log.
2021-06-11 15:19:37,918 - build - log - INFO - HydroMT version: v0.4.2.dev
2021-06-11 15:19:37,918 - build - main - INFO - Building instance of wflow model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full.
2021-06-11 15:19:37,918 - build - main - INFO - User settings:


2021-06-11 15:19:37,962 - build - model_plugins - DEBUG - Loaded model plugin 'wflow = hydromt_wflow.wflow.WflowModel' (0.1.2.dev0)
2021-06-11 15:19:37,962 - build - model_api - INFO - Initializing wflow model from hydromt_wflow (v0.1.2.dev0).
2021-06-11 15:19:37,963 - build - model_api - DEBUG - Setting model config options.
2021-06-11 15:19:37,966 - build - model_api - DEBUG - Default config read from /usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/data/wflow/wflow_sbm.toml
2021-06-11 15:19:37,966 - build - model_api - INFO - setup_basemaps.region: {'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7]}
2021-06-11 15:19:37,966 - build - model_api - INFO - setup_basemaps.res: 0.00833
2021-06-11 15:19:37,966 - build - model_api - INFO - setup_basemaps.hydrography_fn: merit_hydro
2021-06-11 15:19:37,966 - build - model_api - INFO - setup_basemaps.basin_index_fn: merit_hydro_index
2021-06-11 15:19:37,966 - build - model_api - 

2021-06-11 15:19:38,044 - build - data_adapter - INFO - DataCatalog: Getting merit_hydro RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.4/merit_hydro/{variable}.tif


2021-06-11 15:19:38,136 - build - basin_mask - DEBUG - Parsed region (kind=subbasin): {'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7], 'xy': [12.2051, 45.8331]}


2021-06-11 15:19:41,754 - build - basin_mask - INFO - basin bbox: [11.7750, 45.8042, 12.7450, 46.6900] / size: 664756


...

2021-06-11 15:20:23,777 - build - data_adapter - DEBUG - RasterDataset: Convert units for 1 variables.


2021-06-11 15:20:24,264 - build - wflow - INFO - Write model data to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full
2021-06-11 15:20:24,264 - build - model_api - INFO - Writing model config to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full/wflow_sbm.toml
2021-06-11 15:20:24,265 - build - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full/staticmaps.nc


2021-06-11 15:20:24,748 - build - wflow - INFO - Writing model staticgeom to file.


2021-06-11 15:20:24,930 - build - wflow - INFO - Write forcing file
2021-06-11 15:20:24,937 - build - wflow - INFO - Process forcing; saving to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_full/inmaps-era5-2010.nc
[                                        ] | 0% Completed |  0.0s

[########################################] | 100% Completed |  0.1s


With this example we build a complete wflow model including forcing. Compared to the previous **hydromt build** we have added:
* `-i wflow_build.ini` : setup configuration file including all components to build and their arguments

Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow configuration (.toml) file. These files are sufficient to run the wflow model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [6]:
import os
root = 'wflow_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

wflow_test_full
 - hydromt.log
 - staticmaps.nc
 - wflow_sbm.toml
 - inmaps-era5-2010.nc
wflow_test_full/run_default
wflow_test_full/instate
wflow_test_full/staticgeoms
 - rivers.geojson
 - gauges.geojson
 - basins.geojson
 - gauges_grdc.geojson
 - region.geojson
 - glaciers.geojson


### Visualize and/or inspect model schematization

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.